
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/e726eb879f73c25b74029777546f2097b11660d3/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Нейроновости Северной Кореи"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

rukcna.tar.gz       100%[===================>] 443.70M  93.4MB/s    in 5.9s    
Нейроновости Северной Кореи: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [2]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [3]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [ ]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

In [5]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов =   3#@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [6]:
!nvidia-smi

Sun Feb 14 13:05:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [7]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 8.4MB/s 
     |████████████████████████████████| 573kB 9.0MB/s 
     |████████████████████████████████| 890kB 15.2MB/s 
     |████████████████████████████████| 3.7MB 37.3MB/s 
     |████████████████████████████████| 133kB 61.2MB/s 
     |████████████████████████████████| 1.2MB 58.4MB/s 
     |████████████████████████████████| 7.2MB 51.5MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [8]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [9]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [10]:
#@markdown ### Начать генерацию!

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **42**

**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Глава правительственной делегации Кореи на высоком совещании по развитию двустороннего обмена, прошедшем 26 июня в Хабаровске России, заверил в поддержке этого решения. И подчеркнул, чтобы и дальше продолжали тесное и теплое взаимодействие с Кореей в области социального обеспечения в интересах развития и развития двустороннего обмена. Он сказал нижестоящее. И выразил уверенность в том что традиционные отношения между Корейской Народной Армией, Народной Кооператив Южной стороны (АСХ) между КНДР – постоянным партнером двустороннего сотрудничества, добившихся за десятки и сотни лет на Корейском полуострове благодаря уважаемому высшему руководителю товарищу Ким Чен Ыну, укрепляются. <br> Уважаемый высший товарищ товарищ Ким Чен Ын, говоря, что в нынешнем году, когда отмечается 60-летия создания корейско – российско-курдской двухсторонкой комиссии по торговле драгоценным результатом сотрудничества в экономической, научной и образовательной сфере, будет рассматривать вопрос на месте и обсуждать его на месте, подчеркнул, что, прежде чем решить этот вопрос на высшем уровне и обсудить с кем – – либо, следует глубоко знать и уважать стратегическую цель нашей работы. Это наша позиция в отношении развития двустороннего обмена на более высокий уровень. И отметил, как в прошлый раз мы договорились в рамках достигнутой совместной договоренности по сотрудничеству между КНДР и Россией об открытии в нынешнем году нового периода смягчения ситуации, и о развитии отношений, в которых будут участвовать Россия и Китай, на новую стадию в развитии. Он подчеркнул, что и впредь в области двусторонней торговли и сотрудничества будет уделять первостепенное внимание развитию сотрудничества в областях науки, промышленности товаров и морского хозяйства, а в этом направлении – более активно продвигать вперед строительство экономической, культурной, спортивной, туристической и спортивной сфер. <br> Он заверил о том, чтобы активно развивать обмен, в том числе двусторонние отношения между КНДР - Китаем, активно содействовать укреплению сотрудничества на международной арене для выполнения Пханмунчжомской декларации, выполнять свою ответственность за развитие двустороннего сотрудничества и в дальнейшем развивать их вместе с корейским народными правительства. Это наша твердая позиция, и мы не чувствуем ни малейшей уступки, если будем придерживаться позиции неизменности – развивать корейский вопрос и прилагать активные силы к этому, – подчеркнул глава. <br> Он подчеркнул следующее, что будет стараться развивать сотрудничество в областях науки и техники для того, чтобы развивать экономику, и прилагать большие силы к решению проблемы, и активно прилагать усилия к работе, связанной с обеспечением цивилизованных условий, включая экономическую реконструкцию объектов и строительст<br>

**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Премьер РФ Дмитрий ВладимировичПутин по поручению премьера кабинета министров КНДР Пак Бон Чжу 9-го сентября встретился и с министром финансов России Александром Ферингиным обсудил вопрос о повышении с 2019 г. до 2024 г пенсии. <br> На беседе, в котором обе сторона обсудили вопросы по повышению пенсии, приняты решения по увеличению и использованию накопительной части государственного вклада и по вопросу, обсужденным в переговорах в отдельности. Также приняты меры о мерах в отношении разных сфер. И обе стороны обменялись успехами в обсужденных на переговорах и совещаниях областях. <br> Также Путин сказал об активней внедрить систему дистанционного обслуживания и обеспечить народ всей полнотой и надежностью государственных средств и в соответствии с требованием развития реальности, включая политику экономии и денежное обращение. В ходе этого будут последовательно обеспечивать качество всех продуктов питания, включая яйца Фабия, мясо соевыми бобами и яйца Фабие, чтобы не возникло никакой конкуренции с импортных изделий и товаров. Также будут более развивать дистанционные технологии для обеспечения максимальной пользы экономике государства и улучшенной народной жизни. В частности будет активно внедряться информационная и техническая сеть в области здравоохранения, - подчеркнул президент. Он отметил об усиливающем стимулировании развития государственных здравоохранительных органов в целях защиты здоровья населения и обеспечения полной безопасности жизни. <br> На переговорах достигнута договоренность о сотрудничестве между министерствам финансов РФ и правительства КНДР, и подписан документ. В свою очередь, подписан протокол об обмене глубокими техническими мнениями. <br> В свою очередь Путин отметил следующее: В нынешнем мире, где все страны находятся в конфронтации и гонке за господство, невозможно ожидать развития отношений сотрудничества между двумя партиям. И если у нас в обществе, несмотря на злобное требование империализмов и всякие вызовы и трудности и опираясь на собственные силы, последовательно и упорно борются и продвигаются вперед с высоко приподнятыми революционерами в авангарде, то мы не будем и бров с места смотреть, сложат свои оружия на пути к достижению победы в строительстве социализма – таков наш ответ на ожидание империалистов. И мы будем и впредь активно стараться для того же, чтобы в мире не произошел новый кризис из – за вмешательства во всякие дела чужих стран. Это не может стать случайной вещью, а имеет значение и в развитии экономики страны и общества, - отметил российский правитель. <br> Также на встречах Путин сказал: в связи с нынешним положением в России мы с президентом Путиным пришли к единодушенному выводу, что в ближайшее время будут обсуждать вопросы <br>

...

Пхеньян 10 июля в уезде Самчжиён обрушились 2-ые обрушились 2-ьи обрушились 2,500 домов по улице Рёмён, в городе Расан провинции Чаган, провинции Северный Хамген, г. Расан провинции Северной Пхёнъан, г.. Пхунсон провинции Южной Пхёнчон. И обрушилась стена, разрушено здание магазина в гущине города Пхеньян провинции Чаган, и разрушены дома. И разрушено и ограбленно более 100 тыс. жилых домов и общественных зданий г. Пхеньян. <br> Также разрушена часть здания больницы и общежития для иностранцев г. Расал и др. В результате того пострадали более 100 тыс. жилых домов, а число жертв увеличилось на 2 миллиона. В частности в уезде Самчжиёна разрушились здания больницы имени Кан Сун Док в г. Нампхо, больницы имени Кан Сун Док и других больницах провинции Северный Хамгён. И разрушена и ограблено около 200 домов. <br> Как известно всем в Корее, 2 года назад произошло обрушения 3 жилых домов в гуппа провинции Северный Хванвон и г. Самчжиён в уезды Чжанган провинции Северной Чхольцзян и уезд Рёмен провинции Северной Хванхэ. Также обрушены 2 дома гп. Чаган провинции Южной Пхеньян и гтп провинции Южной Хамгён. Также разрушены дома г. Нампхо, провинции Чаган и г. Пхеньянский. <br> По сводным данным, в провинции Южный Хамгён в период 2-го обрушились 3 тыс. 810 квадратных метров и в городе Пхеньян, провинции Северный Хванвон в 1.3 раза, а число жертв достигло более 3 млн 400 семей и более 1.5 млн 400 тыс. людей получили ожоги. <br> -о<br>

**В Казахстане обрушился мост через Вселенную**

Инцидент потопления судна японской разведывательной группой
В последние дни на земле Общенационального заповедник рыб «Ынгым» произошел инцидент потопления судна японской разведывательной групп «Сэгок», в котором приняли в море «Нгуок», которое является главным торговым партнером Японии для осуществления притязания японских реакционеров к «регентам на острове Гуам», за совершение торговли с Японами. <br> Инцидент потопления японского разведывательного судна в акватории Восточного океана в сентябре текущего года произошел по вине разведывательного корабля «Ренъо», совершающего разведывательный рейс по маршруту из Пхеньё до Мусудана Китая, но, не смотря на это японские разведывательные круги не прекращали попытки повторной попытки, в том числе с целью «подавить и уничтожить нашу Республику путем ускорения продвижения процесса создания благоприятного условия для осуществления амбиции захвата островной земли». В результате инцидента, в который вовлекает большое международное общественное влияние Япония, произошел ущерб от удара нашей авиации, и это является серьезной провокацией, угрожающей судьбе и безопасности корейского полуострова, Японии и миру. <br> Как бы в прошлые времена японские разведывательные органы ни прибегали только для осуществления своих агрессивных замысли в отношении Кореи, но сегодня ситуация изменилась и японские разведывательные службы не только пытаются преградить течение мира, но и откровенно совершают происками для захвата территории, включая «операция захвата судна», которая нацелено было направлено на похищенных и похищение корейцами людей. <br> По данным, с конца прошлого года японские реакционеры совершили многие агрессивнические преступления, включая убийство десятков корейских женщин в районе озера Самчжа и похищение десятков детей из детского лагеря и принудительные коллективные изнасилования. Это ясно свидетельствует, как божий день, кто является преступником Японии – крупнейшим военным провокационным преступным органом, совершившим преступление. <br> Сейчас японские реакционерам не до изменения общественного мнения насчет Японии, более того японские разведывательные группы все больше цепляются за военные действия в регионе, включая Корейский восточный морской фронт (Чонрён), и тем совершают военные действия в целях вторжения нашей Республики на эту землю. В такой ситуации только японские правые силы, в авангарде выступающие для свержения нашей Республики и захвата нашей Республики – это является серьезной угрозой миру и безопасности всего региона. <br> Япония – заклятого заклятого врага человечества, которая уже потеряла незыблемые идеалы, и которая уже превратилась не в государств<br>

**В Казахстане обрушился мост через Вселенную**

Недавно, 5 июля на реке Тэдон города Кэчхон провинции Южной Хамген Кореи, произошел большой ополмёт на плоскогорье Босфор, и обрушились многочисленные слои земли на этом районе. И в тот день, 5 июля, произошел крупный ополмитный сдвиг. В провинции Южной Хванхэ в районе Босфор провинции Южной Хванхэ, где образовалось брешь прорыва и бурлила стела взрыва моста над плотиной, образовалось брешь прорыва и обрушены многочисленные слои земли. Также на плоских берегах Босфора, у подножий плотины ГЭС «Армия и революция» провинции Канвон и в районе села Чхончхон провинции Южной Пхёчхон образовалось брешь прорыв и образовался прорыв воды. <br> На пострадавших местах обнаружены останки, а также обнаружены останки более 100 человек и многие детали разрушенных мостов. Это – останки погибших. <br> В провинции Северный Хванхэ, городе Тэгу провинции Ривон тоже был серьезный удар и обрушено немало слоев земли. Также разрушились более 70 мостов, в том районе – десятки ГЭС, а на пострадавших местах образовалось брешь разрыва. <br> На пострадавших от стихийных бедствии регионах в провинциях Северный и Северный Пхеньъан тоже произошли сильные ополмические столкновения из разных стихийно-активных материалов в результате чего разрушились многие мостов, но и их число все еще остается большим, чем раньше или позже после стихийно-аграрного бедствия. И это еще раз ясно свидетельствует то положение, которое в настоящее время усугубляют власти. <br> -о<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [ ]:
%%writefile input.txt


Overwriting input.txt


In [11]:
#@markdown ### Генерируем новую выдачу

Повторов = 3  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **9005**

**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

Председатель Президиума ВНС КНДР Ким Ен Нам 2 декабря в Мансудэском дворце съездов выразил решимость обсудить вопрос об индексации пеней работающим в старости людям, проживающим на селе. И в этот день президент Российской Федерации Владимир Владимирович Путин выступил с исторической речью. <br> Он сказал, что для того, чтобы в стране, которая достойно занимает статус мировой ядерной державой в силу своего превосходного строя, более упрочить материально-техническую базу, нужно последовательно обеспечить условия, в которых трудящиеся наслаждаются жизнью в старости, и тем самым выполнить свою ответственность. <br> Он выразил позицию, что правительство и народ РФ, поддерживая историческую речь уважаемого высшего руководителя на месте для развития отношений дружбы и сплоченности между РФ и КНДР, будут прилагать активные усилия для укрепления сотрудничества в различных областях. <br> В свою очередь, президент Туркменистанской Федеральной Республики Ильхам Алиев, находящийся с визитом в нашей стране, дал указание о том, чтобы в соответствующих регионах последовательно соблюдали принципы взаимозаменяемости, взаимопомощи и взаимопомощи между работниками, включая пенсии. Также отметил необходимость последовательного выполнения Пханмунчжомской декларации и Совместного корейско-российского документа. <br> -о<br>

**Путин пообещал рассмотреть вопрос об индексации пенсий работающим пенсионерам**

В свою очередь врио представителя президента в КНДР Ли Сон Гвон встретился с членом верхней палаты парламента Венесуэлы, президентом Социалистической Республики Венесуэла Беронимо Кастилё и его семьей. <br> Он отметил следующее: <br> В настоящее время в разных странах проводятся дискуссии и совещания по повышению благосостояния населения. <br> Мы с удовольствием поддерживаем и поддерживаем борьбу народа Кореи за улучшение благосостояния народа в ответ на санкции и нажим. <br> В связи с нынешней обстановкой я еще раз выражаю твердую позицию, чтобы правительство Венесуэлы приняло активные меры по повышению благосостояния народа. <br> Мы будем более активизировать обмен и сотрудничество с соответствующими международными органами. <br> -о<br>

...

В районе Пхёнчхоне г. Нампхо обрушилось плотинное сооружение моста «Окрю», и его разрушение ускорило время. <br> Инцидент произошел, как правило, в день строительства. В результате чего плотина, где проникнута угроза взрыва из-за взрыва ядерной войны США на корейском полуострове, разрушается, и в течение 10 дней на этом месте непрерывно образуется вода. И так продолжается из-за ущерба окружающей среде. <br> Сейчас пострадавший от бедствия уездный народный комитет решил созвать экстренную пресс-конференцию для выяснения истины инцидента и принять меры по ликвидации ущерба. И он подчеркнул следующее. <br> По случаю Дня Солнца в уезде Самчжиён, уезде Кёнсон и других местах провинции Северной Пхёньан и провинций Южный и Северный Хамгён проведены митинги горожан и жителей с требованием немедленно принять срочные меры. В этих митингах приняли участие работники и коллектив ГЭС и соответствующих единиц, трудящиеся. <br> На митингах были доклады, затем были прения, затем были танцы молодежи и учащейся молодежи. <br> Выступавшие отметили, что в прошлом году на этом святом месте в течение несколько десятков дней разрушено более 10 плотин, и тем восстановлена водоснабжение, также создана материковая часть Республики, и они будут прилагать большие усилия к ликвидации ущербов, чтобы полностью восстановить пострадавшую плотину в течение десятков дней. <br> -о<br>

**В Казахстане обрушился мост через Вселенную**

В районе Кансо города Караганда Казахстана обрушилось несколько сот тонн земли. И там, где в прошлое время находился мост «Армия и народ», обрушились многочисленные камни и куски черепиц. <br> Как выяснилось, в результате обрушения разрушены несколько зданий, а также разрушены и мосты, по которым текла река «Армия» в районе Кансо, и в районе города Караганда – мост. <br> Сейчас пострадавшие от наводнения и сильного ливня жители, которые остались дома в развалинах из-за стихийных бедствий и бедственной жизни населения, продолжают путь на работу и живут в своих домах. <br> Пострадавшие в провинции Канге жители, не выдерживая вспыхнувшего волнения, сказали, что они, конечно, переживают, что обрушилась мост и это является трагедией, что в такое время они остались дома, и выразили решимость продолжать борьбу до тех пор, пока восстановительная работа по ликвидации последствий не прекращается. <br> -о<br>

**В Казахстане обрушился мост через Вселенную**

В разных регионах страны, включая провинцию Северный Пхёнъан, провинцию Южный Пхёнъан, город Нампхо и город Канрён провинции Северный Хванхэ, произошел взрыв моста через Вселенную. <br> Как только раздался гул взрыва, мост через Вселенную в разных местах обрушилось на плоскогорье Кызыл. По словам местных жителей, взрыв произошел в течение несколько часов, в течение двух дней – более 10 часов, а в некоторых регионах произошел взрыв, и тем прервались движение грузов на плоскогорье. В результате этого, движение грузовиков и поездов в провинции Канрён прервалось, также движение автобусов и велосипедных машин по мосту через вселенную в провинции Южный Хамгён не прекращается и в других провинций и городах тоже. В результате того, движение транспортных средств и поездов в провинции Северный Хванхэ и городе Нампхо прервалось в короткое время и в провинции Северный Хванхэ тоже прекратилось движение. <br> В провинции Южный Хванхэ взрыв моста произошел по причине того, что из-за сильного ливня в провинции Южный Пхёнъан в течение несколько часов подряд шли дожди, и в результате чего движение грузов в провинции Южный Хамгён затруднено и затруднено движение грузовиков и велосипедных транспортных средств. <br> На фоне того происшествия в провинции Южный Хамгён, в городах Канрён и Канрён, в провинциях Канвон и в городе Нампхо и городе Нампхо и провинции Северный Хванхэ взрыв моста через вселенную стал еще одним взрывчатым веществом, разрушившим плоскоерь Кызыл. <br> -о<br>

На этом всё. Получайте удовольствие (надеюсь)!